<a href="https://colab.research.google.com/github/SimonNachtergaele/AI-Track-tive/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Dec 23 10:44:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'06102020 Copernicus template.docx'
'AAP WP bevraging.gdoc'
 AITracktivev1.7.py
'Ana Belgium start-up.gdoc'
'Colab Notebooks'
'Jasper thesis.gdoc'
'My Drive'
'Questionnaire geology November 2020.gform'
'Questionnaire geology November 2020 (Responses).gsheet'
'script v4.py'
 scriptv6.4.py
 scriptv6.py
'Sharmaine thesis.gdoc'
'Untitled form.gform'
 yolov3
 yolov4
 yolov4_training_simon.cfg


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14648 (delta 0), reused 2 (delta 0), pack-reused 14645
Receiving objects: 100% (14648/14648), 13.24 MiB | 23.53 MiB/s, done.
Resolving deltas: 100% (9970/9970), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 8000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [7]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-23 10:46:33--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   446KB/s    in 6m 32s  

2020-12-23 10:53:05 (405 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/classes.txt    
  inflating: data/obj/F115 ed flipped_XY02_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY02_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY03_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY03_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY04_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY04_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY05_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY05_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY07_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY07_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY08_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY08_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY09_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY09_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY10_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY10_Z5.txt  
  inflating: data/obj/F115 ed flipped_XY11_Z5.jpg  
  inflating: data/obj/F115 ed flipped_XY1

In [10]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaminguitvoer ingekort tot de laatste 5000 regels.
15 0.337687 0.730721 0.026119 0.036070

['15', '0.337687', '0.730721', '0.026119', '0.036070']
44 data/obj/F115 ed flipped_XY13_Z5.txt
0 0.337687 0.730721 0.026119 0.036070
15 0.371891 0.771766 0.017413 0.011194

['15', '0.371891', '0.771766', '0.017413', '0.011194']
44 data/obj/F115 ed flipped_XY13_Z5.txt
0 0.371891 0.771766 0.017413 0.011194
15 0.383085 0.794776 0.024876 0.092040

['15', '0.383085', '0.794776', '0.024876', '0.092040']
44 data/obj/F115 ed flipped_XY13_Z5.txt
0 0.383085 0.794776 0.024876 0.092040
15 0.465174 0.873134 0.057214 0.044776

['15', '0.465174', '0.873134', '0.057214', '0.044776']
44 data/obj/F115 ed flipped_XY13_Z5.txt
0 0.465174 0.873134 0.057214 0.044776
15 0.524254 0.917289 0.065920 0.038557

['15', '0.524254', '0.917289', '0.065920', '0.038557']
44 data/obj/F115 ed flipped_XY13_Z5.txt
0 0.524254 0.917289 0.065920 0.038557
15 0.482587 0.956468 0.022388 0.019900

['15', '0.482587', '0.956468', '0.022388'

In [11]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/F115 ed flipped_XY23_Z5.jpg', 'data/obj/F115 ed flipped_XY11_Z5.jpg', 'data/obj/M12_FCT-G4_EDr_crop_XY012_Z4_RGB.jpg', 'data/obj/Seq0000_XY008_Z4_RGB.jpg', 'data/obj/F115 ed flipped_XY10_Z5.jpg', 'data/obj/Seq0000_XY004_Z3_RGB.jpg', 'data/obj/F115 ed flipped_XY26_Z5.jpg', 'data/obj/F115 ed flipped_XY14_Z5.jpg', 'data/obj/M20_DUR-G2_EDr-Uc_crop_XY05_Z4_RGB.jpg', 'data/obj/Seq0000_XY007_Z4_RGB.jpg', 'data/obj/F115 ed flipped_XY15_Z5.jpg', 'data/obj/M20_DUR-G2_EDr-Uc_crop_XY02_Z4_RGB.jpg', 'data/obj/M20_DUR-G2_EDr-Uc_crop_XY03_Z4_RGB.jpg', 'data/obj/M12_FCT-G4_EDr_crop_XY054_Z4_RGB.jpg', 'data/obj/M12_FCT-G4_EDr_crop_XY002_Z4_RGB.jpg', 'data/obj/F115 ed flipped_XY04_Z5.jpg', 'data/obj/M12_FCT-G4_EDr_crop_XY017_Z4_RGB.jpg', 'data/obj/F115 ed flipped_XY19_Z5.jpg', 'data/obj/F115 ed flipped_XY02_Z5.jpg', 'data/obj/F115 ed flipped_XY18_Z5.jpg', 'data/obj/F115 ed flipped_XY27_Z5.jpg', 'data/obj/Seq0000_XY010_Z4_RGB.jpg', 'data/obj/M12_FCT-G4_EDr_crop_XY005_Z4_RGB.jpg', 'data/obj/F11

In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [15]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000487, iou_loss = 0.000000, total_loss = 0.000487 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.019582, iou_loss = 0.000000, total_loss = 0.019582 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.659642), count: 432, class_loss = 47.236729, iou_loss = 37.228252, total_loss = 84.464981 
 total_bbox = 2690121, rewritten_bbox = 1.048726 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000532, iou_loss = 0.000000, total_loss = 0.000532 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.330002), count: 1, class_loss = 0.273954, iou_loss = 0.433264, total_loss = 0.707218 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.612082), count: 592